In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
err = pd.read_csv("train_err_data.csv")
quality = pd.read_csv("train_quality_data.csv")
problem = pd.read_csv("train_problem_data.csv")

## train_err_data
(데이콘 참고) 고객이 식기 세척기를 사용하는 중에 오류가 발생하면 상태와 관련 있는 로그만 자동으로 고객서비스 센터로 전송합니다. 이 오류 자동 검출 정보가 train_err_data.csv 입니다.
- 고객번호(user_id) : 제품등록 및 동의 과정에서 자동생성한 Unique한 고객번호
- 오류발생일시(time) : 식기세척기 오류가 발생한 일시
- 모델(model_nm) : 식기세척기 모델명
- 식기세척기 모델 Firmware 버전(fwver) : 식기 세척기를 작동 시키는 내부 프로그램 버전
- 오류유형(errtype) : 네트워크 오류, 전원 오류 등 약 40여종으로 구분되는 오류 유형(대분류)
- 오류유형별 코드(errcode) : 발생된 오류 유형별 코드로 중분류

In [3]:
err.shape

(16554663, 6)

In [4]:
err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [5]:
err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [6]:
print(err.isnull().sum())
err.dropna(axis=0, inplace=True)

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64


In [7]:
err.drop('time', axis=1, inplace=True)

In [8]:
print(err.groupby(['model_nm','fwver']).size())

err.drop('model_nm', axis=1, inplace=True)

model_nm  fwver     
model_0   04.22.1442       2522
          04.22.1656         39
          04.22.1666          5
          04.22.1684       5554
          04.22.1750    2874213
          04.22.1778    1293946
model_1   04.16.2641        296
          04.16.3345        101
          04.16.3439        142
          04.16.3553    5237816
          04.16.3569        980
          04.16.3571     145156
model_2   04.33.1095        909
          04.33.1125        408
          04.33.1149       3272
          04.33.1171         58
          04.33.1185     963736
          04.33.1261    2504870
model_3   05.15.2090         12
          05.15.2092         18
          05.15.2114          9
          05.15.2120        706
          05.15.2122         23
          05.15.2138    1906479
          05.15.3104        191
model_4   03.11.1141         29
          03.11.1149       2114
          03.11.1167    1505659
model_5   04.82.1684      23554
          04.82.1730         78
          04.82.177

In [9]:
err.errtype.unique()

array([15, 12, 11, 16,  4, 26, 10, 14,  3, 13,  6,  7, 27, 28, 31, 33, 22,
       41,  5, 40, 34, 37, 36, 23, 18, 20, 19, 21, 42, 24, 25, 17, 38, 39,
       35, 32, 30,  2,  1,  9,  8], dtype=int64)

In [10]:
err_num = err.groupby(['errtype', 'fwver']).size().sort_values(ascending=False)
err_num.describe()

count    7.270000e+02
mean     2.277120e+04
std      1.051766e+05
min      1.000000e+00
25%      6.000000e+00
50%      6.400000e+01
75%      1.539000e+03
max      1.468296e+06
dtype: float64

In [11]:
err_num

errtype  fwver     
23       04.16.3553    1468296
32       04.16.3553    1073268
4        05.15.2138     997780
22       04.16.3553     970609
28       03.11.1167     538250
                        ...   
19       04.82.1684          1
         04.73.2571          1
5        04.22.1656          1
19       04.33.1149          1
5        04.33.1125          1
Length: 727, dtype: int64

## train_quality_data
(데이콘 참고)사용자가 식기 세척기를 작동하면서 오류가 발생하면 측정가능한 지표 12가지를 2시간 간격으로 수집하여 자동으로 LG 고객 서비스 센터로 전송됩니다. 이렇게 수집된 정보 828,624건이 제공됩니다.
- 오류 로그 생성일시(time), 
- 식시 세척기 보유 고객번호(user_id), 
- 식기세척기 모델 Firmware 버전(fwver),
- 식시 세척기의 12가지 오류 상태의 발생횟수(quality_0 ~ quality_1)

In [12]:
quality.shape

(828624, 16)

In [13]:
quality.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [14]:
quality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   time        828624 non-null  int64  
 1   user_id     828624 non-null  int64  
 2   fwver       788544 non-null  object 
 3   quality_0   684192 non-null  float64
 4   quality_1   828624 non-null  int64  
 5   quality_2   788511 non-null  float64
 6   quality_3   828624 non-null  int64  
 7   quality_4   828624 non-null  int64  
 8   quality_5   828604 non-null  object 
 9   quality_6   828624 non-null  int64  
 10  quality_7   828624 non-null  object 
 11  quality_8   828624 non-null  object 
 12  quality_9   828624 non-null  object 
 13  quality_10  828624 non-null  object 
 14  quality_11  828624 non-null  int64  
 15  quality_12  828624 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 101.2+ MB


In [15]:
quality.drop('time', axis=1, inplace=True)

In [16]:
quality.drop_duplicates(inplace=True)

In [17]:
quality.isnull().sum()

user_id           0
fwver          8310
quality_0     20312
quality_1         0
quality_2      8343
quality_3         0
quality_4         0
quality_5        15
quality_6         0
quality_7         0
quality_8         0
quality_9         0
quality_10        0
quality_11        0
quality_12        0
dtype: int64

In [18]:
x = quality[quality['quality_2'].isnull()]
x[x['fwver'].isnull()]
# fwver가 결측치인값 모두 quality_2가 결측치

,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
38316,10693,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0
60168,11206,NaN,NaN,1,NaN,0,0,0,0,0,3,0,0,0,0
60169,11206,NaN,NaN,2,NaN,0,0,0,0,0,3,0,0,0,0
60170,11206,NaN,NaN,0,NaN,0,0,0,0,0,3,0,0,0,0
60171,11206,NaN,NaN,-1,NaN,0,0,-1,-1,0,3,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828313,24982,NaN,NaN,0,NaN,0,0,5,0,0,0,0,16,0,0
828314,24982,NaN,NaN,0,NaN,0,0,0,0,0,0,0,16,0,0
828315,24982,NaN,NaN,-1,NaN,0,0,-1,-1,0,0,0,16,-1,0
828324,24982,NaN,NaN,0,NaN,0,0,0,0,0,0,0,0,0,0


## train_problem_data
(데이콘 참고)오프라인 매장에 있는 LG 고객서비스 센터 센터에 불만/불편이 접수된 정보인 train_problem_data.csv 가 제공됩니다. 이 파일에는 고객번호(user_id), 불편/불만 발생일시(time)이 제공됩니다.
- 1명의 고객이 여러번 불편/불만을 접수했을 수 있으며, 
- 식기 세척기 오류 자동 검출 정보(train_err_data.csv) 파일에 없는 고객도 불편/불만이 접수될 수 있으며,
- 반대로 식기 세척기 오류 자동 검출 정보(train_err_data.csv)에 오류 기록이 있고 불편/불만 접수를 하는 고객도 있습니다.

In [19]:
problem.shape

(5429, 2)

In [20]:
problem.head()

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


In [21]:
problem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5429 entries, 0 to 5428
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  5429 non-null   int64
 1   time     5429 non-null   int64
dtypes: int64(2)
memory usage: 85.0 KB


In [22]:
problem['time'] = problem['time'].astype(str)

In [23]:
problem['time'] = pd.to_datetime(problem['time'])

In [24]:
problem['time_month'] = problem['time'].dt.month
problem['time_day'] = problem['time'].dt.day
problem.drop('time', axis=1, inplace=True)
problem

,user_id,time_month,time_day
0,19224,11,2
1,23664,11,16
2,15166,11,14
3,12590,11,8
4,15932,11,3
...,...,...,...
5424,20167,11,25
5425,16270,11,10
5426,19114,11,6
5427,21505,11,4


In [25]:
pd.merge(problem, quality, how='left', on='user_id')

,user_id,time_month,time_day,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,19224,11,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,23664,11,16,04.22.1750,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,3,0.0,0.0
2,23664,11,16,04.22.1750,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,3,0.0,0.0
3,23664,11,16,04.22.1750,0.0,0.0,0.0,0.0,0.0,2,0.0,0,0,0,3,0.0,0.0
4,23664,11,16,04.22.1750,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159341,18822,11,2,05.15.2138,0.0,0.0,0.0,0.0,0.0,5,0.0,0,0,0,7,0.0,0.0
159342,18822,11,2,05.15.2138,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,7,0.0,0.0
159343,18822,11,2,05.15.2138,0.0,0.0,0.0,0.0,0.0,1,0.0,0,0,0,4,0.0,0.0
159344,18822,11,2,05.15.2138,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,4,0.0,0.0
